# Step 4: Post-Processing function

In [ ]:
import os, sys, time, importlib, glob
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
sys.path.append("../../../GOSTnets")
import GOSTnets as gn

# pip install osmium
# import osmium, logging
# import shapely.wkb as wkblib

from shapely.geometry import LineString, Point

In [ ]:
# This is a Jupyter Notebook extension which reloads all of the modules whenever you run the code
# This is optional but good if you are modifying and testing source code
%load_ext autoreload
%autoreload 2

In [ ]:
from GOSTnets.load_traffic2 import *

In [ ]:
# read graph
G = nx.read_gpickle('./sri_lanka_unclean2_w_time_largest_20200616.pickle')

In [ ]:
len(G.edges)

In [ ]:
for node in G.neighbors(7063613803):
    print(node)

In [ ]:
G[258522955]

In [ ]:
# start time
start = time.time()

In [ ]:
path = r'./output_edges_shapefiles_20200529' # use your path
all_files = glob.glob(path + "/*.shp")

li = []

for filename in all_files:
    gdf = gpd.read_file(filename)
    li.append(gdf)

study_area = gpd.GeoDataFrame(pd.concat( li, ignore_index=True))

In [ ]:
# import shapefile as GPD
#study_area = gpd.read_file("./output_edges_shapefiles_20200529/weighted_sec_saved_edges_1243386867.shp")

In [ ]:
study_area

In [ ]:
study_area['o'] = study_area['o'].astype(int)
study_area['d'] = study_area['d'].astype(int)           

In [ ]:
study_area.dtypes

### Adding the extra attributes to the graph

In [ ]:
def modify_graph(G, table):
    for i in range(len(table)):
    #for i in range(3):
        row_dict = study_area.iloc[i].to_dict()
        #print('print row_dict o')
        #print(row_dict['o'])
        #print('print row_dict d')
        #print(row_dict['d'])
        for key in row_dict:
            if not key == 'o' and not key == 'd':
                #print(f'print key: {key}')
                G[row_dict['o']][row_dict['d']][0][key] = row_dict[key]
                G[row_dict['d']][row_dict['o']][0][key] = row_dict[key]
        

In [ ]:
modify_graph(G, study_area)

In [ ]:
# objective: expand bracket section by looking at neighbors and adding if length is less than the combined length
# of the bracketed section

In [ ]:
def combine_and_replace_path(path):
    global G
    global final_table
    
    all_nodes_to_remove = []
    paths_to_simplify = []
    
    print('print path inside combine_and_replace_path function')
    print(path)
    
    for index, node in enumerate(path):
        #print(node)
        try:
            print(path[index])
            print(path[index+1])
            paths_to_simplify.append([path[index], path[index+1]])
        except:
            print('end')
    
    print('print paths_to_simplify')
    print(paths_to_simplify)

    edge_attributes = {}
    
    for item in paths_to_simplify:
        #print('print path[:-1]')
        #print(path[:-1])
        #print('print path[1:]')
        #print(path[1:])
        for u, v in zip(item[:-1], item[1:]):
            edge = G.edges[u, v, 0]
            # look in GeoDataFrame instead
            #edge = study_area_copy.loc[(study_area_copy.o == str(u)) & (study_area_copy.d == str(v))]
            print('print edge')
            print(edge)
            for key in edge:
                if key in edge_attributes:
                    # if this key already exists in the dict, append it to the
                    # value list
                    edge_attributes[key].append(edge[key])
                else:
                    # if this key doesn't already exist, set the value to a list
                    # containing the one value
                    edge_attributes[key] = [edge[key]]

    print('print edge_attributes before')
    print(edge_attributes)
    
    for key in edge_attributes:
        # don't touch the length, sec_saved, weighted_s, or mean_speed values
        if key == 'geometry':
            edge_attributes['geometry'] = list(edge_attributes['geometry'])
        #elif key == 'o':
            #edge_attributes[key] = edge_attributes[key][0]
        #elif key == 'd':
            #edge_attributes[key] = edge_attributes[key][-1]
        elif key != 'length' and key != 'geometry' and key != 'sec_saved' and key != 'weighted_s' and key != 'mean_speed':      # if len(set(edge_attributes[key])) == 1 and not key == 'length':
            # if there's only 1 unique value in this attribute list,
            # consolidate it to the single value (the zero-th)
            edge_attributes[key] = edge_attributes[key][0]
        elif not key == 'length' or key == 'sec_saved' or key == 'weighted_s' or key == 'mean_speed':
            # otherwise, if there are multiple values, keep one of each value
            edge_attributes[key] = list(set(edge_attributes[key]))
    # construct the geometry and sum the lengths of the segments
    #edge_attributes['geometry'] = LineString([Point((G.nodes[node]['x'], G.nodes[node]['y'])) for node in path])

    try:
        print('print edge_attributes before2')
        print(edge_attributes)
        #print('before loop1')
        #need to union the geometries
        edge_attributes['geometry'] = edge_attributes['geometry'][0].union(edge_attributes['geometry'][1])
        #edge_attributes['geometry'] = LineString([G.nodes[node]['geometry'] for node in path])
        ##edge_attributes["geometry"] = LineString([Point((G.nodes[node]["x"], G.nodes[node]["y"])) for node in path])
        #print('before loop2')
        edge_lengt_sum = sum(edge_attributes['length'])
        #print('before loop3')
        edge_attributes['sec_saved'] = sum(edge_attributes['sec_saved'])
        #print('before loop4')
        edge_attributes['weighted_s'] = sum(edge_attributes['weighted_s'])
        #print('before loop5')
        weighted_sum = 0
        #print('before loop')
        for index,item in enumerate(edge_attributes['mean_speed']):
            #print('in loop')
            #print('index')
            #print(index)
            #print('item')
            #print(item)
            #print('edge attributes for length')
            #print(edge_attributes['length'][index])
            weighted_sum += item * (edge_attributes['length'][index] / edge_lengt_sum)
        edge_attributes['mean_speed'] = weighted_sum
        edge_attributes['length'] = edge_lengt_sum
    except:
      print('failed to combine edges, print nodes')
      for node in path:
          print(node)

    print('print path inside combine_and_replace_path function 2')
    print(path)
    all_nodes_to_remove.extend(path[1:-1])
    print('print all_nodes_to_remove')
    print(all_nodes_to_remove)
    
    print('print new edges')
    print(paths_to_simplify[0][0])
    print(paths_to_simplify[1][1])
    
    #add new edges
    G.add_edge(paths_to_simplify[0][0], paths_to_simplify[1][1], **edge_attributes)
    G.add_edge(paths_to_simplify[1][1], paths_to_simplify[0][0], **edge_attributes)
    
    # finally remove all the interstitial nodes between the new edges
    G.remove_nodes_from(set(all_nodes_to_remove))
    
    # from final table 
    # remove edges that make up the path from the GeoDataFrame
    for index, node in enumerate(path):
        #print(node)
        try:
            print('dropping from final table')
            print(path[index])
            print(path[index+1])
            all_edges_to_remove.append([path[index], path[index+1]])
            final_table.drop(final_table.loc[(final_table.o == path[index]) & (final_table.d == path[index+1])].index, inplace=True)
        except:
            print('end')
        #try:
            #all_edges_to_remove.append([path[index+1], path[index]])
            #final_table.drop(final_table.loc[(final_table.o == str(path[index+1])) & (final_table.d == str(path[index]))].index, inplace=True)
        #except:
            #print('end2')

    # Add the new path to the final_table
    print('print edge_attributes final')
    print(edge_attributes)
    final_edge_dict = edge_attributes.copy()
    final_edge_dict['o'] = paths_to_simplify[0][0]
    final_edge_dict['d'] = paths_to_simplify[1][1]
    all_edges_to_add.append([paths_to_simplify[0][0], paths_to_simplify[1][1]])
    final_table = final_table.append(final_edge_dict, ignore_index=True)
    
    
    return final_edge_dict

In [ ]:
def build_path(node, gap_count, path, path_dict, direction):
    global G
    print('print direction')
    print(direction)
    
    try:
        # look at neighbors
        for successor in G.successors(path[direction]):
            if successor not in path:
                print(f'degree of successor: {G.degree(successor)}')
                if G.degree(successor) < 5:
                #if is_endpoint(G, successor, strict = True) == False:
                    print('not an endpoint!')
                    if 'sec_saved' in G.get_edge_data(node, successor)[0]:
                        if G.get_edge_data(node, successor)[0]['sec_saved'] > 0:
                            if direction == 0:
                                path.insert(0,successor)
                                print('print path result2 from origin')
                                print(path)
                                #combine and create new path
                                current_path_dict = combine_and_replace_path(path)
                                print('print path_dict from origin2')
                                print(current_path_dict)
                                path = [current_path_dict['o'], current_path_dict['d']]
                                path = build_path(successor, gap_count, path, current_path_dict, direction = 0)
                            else:
                                path.append(successor)
                                print('print path result2 from destination')
                                print(path)
                                #combine and create new path
                                current_path_dict = combine_and_replace_path(path)
                                print('print path_dict from destination2')
                                print(current_path_dict)
                                path = [current_path_dict['o'], current_path_dict['d']]
                                path = build_path(successor, gap_count, path, current_path_dict, direction = 1)
                            print('print path after combine2')
                            print(path)
                        else:
                            gap_count += 1
                            pass
                    else:
                        if gap_count < 2:
                            gap_count += 1
                            path = build_path(successor, gap_count, path, current_path_dict, direction = direction)
                        #old code  
                        #if edge with traffic add if length is less than bracketed section       
                        #elif G.get_edge_data(node, successor)[0]['length'] < current_path_dict['length']:
                            
    except:
        print('indented block2')
        return path
    return path

In [ ]:
final_table = study_area.copy()

In [ ]:
def is_endpoint(G, node, strict = True):
        """
        Return True if the node is a "real" endpoint of an edge in the network, \
        otherwise False. OSM data includes lots of nodes that exist only as points \
        to help streets bend around curves. An end point is a node that either: \
        1) is its own neighbor, ie, it self-loops. \
        2) or, has no incoming edges or no outgoing edges, ie, all its incident \
            edges point inward or all its incident edges point outward. \
        3) or, it does not have exactly two neighbors and degree of 2 or 4. \
        4) or, if strict mode is false, if its edges have different OSM IDs. \

        Parameters
        ----------
        G : networkx multidigraph

        node : int
            the node to examine
        strict : bool
            if False, allow nodes to be end points even if they fail all other rules \
            but have edges with different OSM IDs

        Returns
        -------
        bool
        """
        
        neighbors = set(list(G.predecessors(node)) + list(G.successors(node)))
        n = len(neighbors)
        d = G.degree(node)

        # print('print neighbors')
        # print(neighbors)

        if node in neighbors:
            # if the node appears in its list of neighbors, it self-loops. this is
            # always an endpoint.
            return 'node in neighbours'

        # if node has no incoming edges or no outgoing edges, it must be an endpoint
        #elif G.out_degree(node)==0 or G.in_degree(node)==0:
            #return 'no in or out'

        #elif not (n == 2 and (d == 2 or d == 4)):
        elif not (n == 2 and (d == 2 or d == 4)):
            # else, if it does NOT have 2 neighbors AND either 2 or 4 directed
            # edges, it is an endpoint. either it has 1 or 3+ neighbors, in which
            # case it is a dead-end or an intersection of multiple streets or it has
            # 2 neighbors but 3 degree (indicating a change from oneway to twoway)
            # or more than 4 degree (indicating a parallel edge) and thus is an
            # endpoint
            return 'condition 3'

        elif not strict:
            # non-strict mode
            osmids = []

            # add all the edge OSM IDs for incoming edges
            for u in G.predecessors(node):
                for key in G[u][node]:
                    osmids.append(G.edges[u, node, key]['osmid'])

            # add all the edge OSM IDs for outgoing edges
            for v in G.successors(node):
                for key in G[node][v]:
                    osmids.append(G.edges[node, v, key]['osmid'])

            # if there is more than 1 OSM ID in the list of edge OSM IDs then it is
            # an endpoint, if not, it isn't
            return len(set(osmids)) > 1

        else:
            # if none of the preceding rules returned true, then it is not an endpoint
            return False

In [ ]:
all_edges_to_remove = []
all_edges_to_add = []

for i in range(len(study_area)):
#for i in range(5):
    path_dict = study_area.iloc[i].to_dict()
    #print(path_dict)
    gap_count = 0

    path = [path_dict['o'], path_dict['d']]
    print('print initial path')
    print(path)
    
    # check if path is in Graph
    try:
        print(G[path_dict['o']][path_dict['d']])
    except:
        print('path is not in graph, pass')
        continue
    
    if 'sec_saved' in G.get_edge_data(path[0], path[-1])[0]:
        if G.get_edge_data(path[0], path[-1])[0]['sec_saved'] > 0:
            try:
                for successor in G.neighbors(path[0]):
                    print(f'print successor: {successor}')
                    if successor not in path:
                        print(f'print successor again: {successor}')
                        print(f'degree of successor: {G.degree(successor)}')
                        if (G.degree(successor) < 5) and (G.degree(path[0]) < 5):
                        #if is_endpoint(G, successor, strict = True) == False:
                            print('not an endpoint!')
                            if 'sec_saved' in G.get_edge_data(path[0], successor)[0]:
                                if G.get_edge_data(path[0], successor)[0]['sec_saved'] > 0:
                                    path.insert(0,successor)
                                    #combine and create new path
                                    current_path_dict = combine_and_replace_path(path)
                                    print('print path_dict from origin')
                                    print(current_path_dict)
                                    path = [current_path_dict['o'], current_path_dict['d']]
                                    print('print path after combine')
                                    print(path)
                                    path = build_path(successor, gap_count, path, current_path_dict, direction = 0)
                                    print('print path result after build_path')
                                    print(path)
                                else:
                                    gap_count += 1
                                    pass
                            else:
                                # jump gap
                                if gap_count < 2:
                                    gap_count += 1
                                    path.insert(0,successor)
                                    #combine and create new path
                                    current_path_dict = combine_and_replace_path(path)
                                    print('print path_dict from origin')
                                    print(current_path_dict)
                                    path = [current_path_dict['o'], current_path_dict['d']]
                                    print('print path after combine')
                                    print(path)
                                    path = build_path(successor, gap_count, path, current_path_dict, direction = 0)
                for successor in G.neighbors(path[-1]):
                    print(f'print successor2: {successor}')
                    if successor not in path:
                        # double-check this?
                        print(f'degree of successor: {G.degree(successor)}')
                        if (G.degree(successor) < 5) and (G.degree(path[1]) < 5):
                            print('not an endpoint!')
                            if 'sec_saved' in G.get_edge_data(path[1], successor)[0]:
                                if G.get_edge_data(path[1], successor)[0]['sec_saved'] > 0:
                                    path.append(successor)
                                    #combine and create new path
                                    current_path_dict = combine_and_replace_path(path)
                                    print('print path_dict from origin')
                                    print(current_path_dict)
                                    path = [current_path_dict['o'], current_path_dict['d']]
                                    print('print path after combine')
                                    print(path)
                                    path = build_path(successor, gap_count, path, current_path_dict, direction = 1)
                                    print('print path result after build_path')
                                    print(path)
                                else:
                                    gap_count += 1
                                    pass
                            else:
                                # jump gap
                                if gap_count < 2:
                                    gap_count += 1
                                    path.append(successor)
                                    #combine and create new path
                                    current_path_dict = combine_and_replace_path(path)
                                    print('print path_dict from origin')
                                    print(current_path_dict)
                                    path = [current_path_dict['o'], current_path_dict['d']]
                                    print('print path after combine')
                                    print(path)
                                    path = build_path(successor, gap_count, path, current_path_dict, direction = 1)
            except:
                print('indented block')
                continue

In [ ]:
for node in G.neighbors(7063613803):
    print(node)

In [ ]:
print(time.time() - start)

In [ ]:
len(all_edges_to_remove)

In [ ]:
len(all_edges_to_add)

In [ ]:
final_table.loc[(final_table.o == 6537961778) & (final_table.d == 7069355066)]

In [ ]:
final_table.loc[(final_table.o == 6537961778) & (final_table.d == 7069355066)]

In [ ]:
final_table

In [ ]:
final_table.to_file(driver = 'ESRI Shapefile', filename = "output_edges_shapefiles_merged_20200616/weighted_sec_saved_edges_all_merged2.shp")